In [1]:
import streamlit as st
from PIL import Image
import numpy as np
import cv2
import time

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
# output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load the COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def load_image(image_file):
    img = Image.open(image_file)
    return img

def detect_objects(image):
    height, width, channels = image.shape
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return class_ids, confidences, boxes, indexes

st.title("YOLO Object Detection")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = np.array(load_image(uploaded_file))
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Detecting objects...")

    if st.button('Detect Objects'):
        start_time = time.time()
        class_ids, confidences, boxes, indexes = detect_objects(image)
        detection_time = time.time() - start_time

        st.write(f"Detection took {detection_time:.2f} seconds")

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                st.write(f"{label}: {confidence:.2f}")

        st.image(image, caption='Processed Image with Detections.', use_column_width=True)


2024-06-21 17:25:19.356 
  command:

    streamlit run c:\Users\dell\.conda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
import streamlit as st
from PIL import Image
import cv2
import numpy as np

def analyze_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, threshold1=30, threshold2=100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    components = [f"Component {i+1}" for i in range(len(contours))]
    return components

st.title("Image Component Analyzer")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    if st.button('Analyze Image'):
        opencv_image = np.array(image.convert('RGB'))
        components = analyze_image(opencv_image)
        
        st.write("Components detected:")
        for component in components:
            st.write(component)
